# WELCOME

In [1]:
# import here
import os
import sys
sys.path.append("../")
from src.pytorch_cl_vae.model import ResNet18Classifier 
from sklearn.metrics import accuracy_score
import torch
from torch.nn import CrossEntropyLoss
from matplotlib import pyplot as plt
%matplotlib inline
# params here
params = {
    'impath' : "C:\Study\classifying_autoencoders\data\input\img_align_celeba\img_align_celeba",
    'attpath' : "C:\Study\classifying_autoencoders\data\input/list_attr_celeba.txt",
    'attributes': ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young'],
    'classes_dim': [2,2,2,2,2],
    'learning_rate': 5e-4,
    'num_epochs': 10,
    'batch_size': 128,
    'image_size': 224
}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 1 - Load Dataset

In [2]:
from src.loader import get_loader
from src.loader import chew

data_loader = get_loader(params['impath'],
                  params['attpath'],
                  params['attributes'],
                  batch_size=params['batch_size'],
                  image_size=params['image_size'])
# xs, ys = next(iter(dset))


Finished preprocessing the CelebA dataset...


## 2 - Create Classifiers

In [3]:
classifiers = []
optimizers = []
for idx, (name, classes_dim) in enumerate(zip(params['attributes'], params['classes_dim'])):
    classifier = ResNet18Classifier(label_dim=params['classes_dim'][idx]).to(device)
    optimizer = torch.optim.Adam(classifier.parameters(), lr=params['learning_rate'])
    
    classifiers.append(classifier)
    optimizers.append(optimizer)

## 3 - Train 

In [ ]:
num_train_samples = len(data_loader)

train_step_i = 0
train_losses = []
train_accuracies = []
for epoch in range(params['num_epochs']):
    for x_batch, ws_batch in data_loader:
        ws_batch = chew(ws_batch)
        train_step_i += 1
        losses = []
        accuracies = []
        for label_index, (classifier, optim) in enumerate(zip(classifiers, optimizers)):
            x_batch = x_batch.to(device)
            optim.zero_grad()
#             classifier.train()
            predictions, _ = classifier(x_batch)
            w_true = ws_batch[label_index].long()
#             w_true = w_true.max(1)[1]
            labels = w_true.max(1)[1].squeeze().to(device)
            loss = CrossEntropyLoss()(predictions, labels)
            loss.backward()
            optim.step()
            losses.append(loss.item())
            labels = w_true.max(1)[1].squeeze()
            labels_predict = predictions.max(1)[1].squeeze().detach().cpu().numpy()
            acc = accuracy_score(labels, labels_predict)
            accuracies.append(acc)
            
        train_losses.append(losses)
        train_accuracies.append(accuracies)
        loss_dict = dict(zip(params['attributes'], losses))
        acc_dict = dict(zip(params['attributes'], accuracies))
        print("\r|progress: {:.2f}% | train step: {} | losses: {} | accuracies: {} |".format(
            100.* train_step_i / (num_train_samples // params['batch_size'] * params['num_epochs']), train_step_i,
                  loss_dict, acc_dict), end='')
        if train_step_i % 100 == 0:
            print()

|progress: 200.00% | train step: 100 | losses: {'Black_Hair': 0.4187580347061157, 'Blond_Hair': 0.1921710968017578, 'Brown_Hair': 0.43640783429145813, 'Male': 0.14870679378509521, 'Young': 0.3842095136642456} | accuracies: {'Black_Hair': 0.796875, 'Blond_Hair': 0.921875, 'Brown_Hair': 0.78125, 'Male': 0.984375, 'Young': 0.8125} |8125} |||
|progress: 400.00% | train step: 200 | losses: {'Black_Hair': 0.32654911279678345, 'Blond_Hair': 0.15924784541130066, 'Brown_Hair': 0.40704581141471863, 'Male': 0.23876097798347473, 'Young': 0.5028913617134094} | accuracies: {'Black_Hair': 0.875, 'Blond_Hair': 0.921875, 'Brown_Hair': 0.7890625, 'Male': 0.890625, 'Young': 0.8125} |5} |} | ||
|progress: 512.00% | train step: 256 | losses: {'Black_Hair': 0.20261931419372559, 'Blond_Hair': 0.16559374332427979, 'Brown_Hair': 0.3389347195625305, 'Male': 0.16671590507030487, 'Young': 0.43385428190231323} | accuracies: {'Black_Hair': 0.921875, 'Blond_Hair': 0.921875, 'Brown_Hair': 0.859375, 'Male': 0.9375, 'Y

## 4 - Plot results

In [ ]:
train_losses = np.array(train_losses)
train_accuracies = np.array(train_accuracies)

for i, attr in enumerate(params['attributes']):
    plt.plot(train_losses[:, i])
    plt.title(attr)
    plt.ylabel('loss')
    plt.show()
    
    plt.plot(train_accuracies[:, i])
    plt.title(attr)
    plt.ylabel('accuracy')
    plt.show()

## 5 - Save models